# Installs

In [ ]:
%pip install lightning
%pip install torchvision
%pip install tensorboard

# Imports

In [ ]:
import lightning.pytorch as pl
import torch
import os
import torchmetrics
from torch import nn
from torch.utils.data import Dataset, ConcatDataset
from lightning.pytorch.loggers import TensorBoardLogger

# Constants

In [ ]:
CLASS_LABELS = ('cel', 'cla', 'flu', 'gac', 'gel', 'org',
                'pia', 'sax', 'tru', 'vio', 'voi')

# Dataset

In [ ]:
class TrainingDataset(Dataset):
    '''Dataset for loading given training examples - examples for each label
    are stored in separate directory'''
    def __init__(self, source_dir: str):
        self.class_dir_paths = [os.path.join(source_dir, c)
                                for c in CLASS_LABELS]
        self.class_dirs = [[os.path.join(c_path, f)
                            for f in os.listdir(c_path)]
                           for c_path in self.class_dir_paths]
        self.class_sizes = [len(c_dir) for c_dir in self.class_dirs]
        self.size = sum(self.class_sizes)

    def __len__(self) -> int:
        return self.size

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        if idx >= self.size:
            raise IndexError('Item index out of range!')

        file_idx = idx
        for class_idx in range(len(CLASS_LABELS)):
            if file_idx < self.class_sizes[class_idx]:
                break
            else:
                file_idx -= self.class_sizes[class_idx]

        data = torch.load(self.class_dirs[class_idx][file_idx])
        data.requires_grad = False
        return data, torch.Tensor([1 if i == class_idx else 0 for i in range(len(CLASS_LABELS))])

In [ ]:
class TensorDataset(Dataset):
    '''Dataset that loads examples and labels stored as tensor files in
    source_dir. Each pair (example, label) is stored as a single .pt file'''
    def __init__(self, source_dir: str):
        self.files = [os.path.join(source_dir, f) for f in os.listdir(source_dir)]

    def __len__(self) -> int:
        return len(self.files)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        data, label = torch.load(self.files[idx])
        data.requires_grad = False
        return data, label

In [ ]:
class Cache(Dataset):
    def __init__(self, source_dataset: Dataset,
                 calculate_label_distributions: bool = False):
        self.data = []
        if calculate_label_distributions:
            label_sum = torch.zeros((len(CLASS_LABELS)))
            for i in range(len(source_dataset)):
                example, label = source_dataset[i]
                label_sum += label
                self.data.append((example, label))
            self.label_distributions = label_sum / len(source_dataset)
        else:
            self.label_distributions = None
            for i in range(len(source_dataset)):
                self.data.append(source_dataset[i])

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.data[idx]

# Model

## Loss function definition

In [ ]:
class WeightedBCELoss(nn.Module):
    '''Binary Cross Entropy that takes into account within-label imbalances'''
    def __init__(self, label_distribution, reduction='mean'):
        super().__init__()
        self.fn_weights = 1 - label_distribution
        self.fp_weights = label_distribution
        self.bce_loss = nn.BCELoss(reduction='none')
        self.reduction = reduction

    def forward(self, input, target):
        loss = self.bce_loss(input, target)
        fn_loss = loss * self.fn_weights * target
        fp_loss = loss * self.fp_weights * (1 - target)
        loss = fn_loss + fp_loss
        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()
        return loss

## Model definition

In [ ]:
class LastLayer(pl.LightningModule):
    def __init__(self, loss: nn.Module, num_features: int = 2048,
                 num_labels: int = len(CLASS_LABELS),
                 learning_rate: float = 1e-3):
        super().__init__()
        self.save_hyperparameters()

        self.learning_rate = learning_rate
        self.fc = nn.Linear(num_features, num_labels)
        self.sigmoid = nn.Sigmoid()

        self.metrics = {
            'loss': loss,
            'accuracy': torchmetrics.Accuracy(task='multilabel',
                                              num_labels=num_labels),
            'precision': torchmetrics.Precision(task="multilabel",
                                                num_labels=num_labels),
            'recall': torchmetrics.Recall(task='multilabel',
                                          num_labels=num_labels),
            'hamming_distance': torchmetrics.HammingDistance(task='multilabel',
                                                             num_labels=num_labels)
        }

    def _get_metrics(self, preds, target, label):
        metrics = {}
        for k, v in self.metrics.items():
            metrics[label+'_'+k] = v(preds, target)
        return metrics

    def forward(self, x):
        x = self.fc(x)
        return self.sigmoid(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        metrics = self._get_metrics(y_hat, y, 'train')
        self.log_dict(metrics, on_step=False, on_epoch=True)

        return metrics['train_loss']

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        metrics = self._get_metrics(y_hat, y, 'validation')
        self.log_dict(metrics, on_step=False, on_epoch=True)

        return metrics['validation_loss']

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        metrics = self._get_metrics(y_hat, y, 'test')
        self.log_dict(metrics, on_step=False, on_epoch=True)

        return metrics['test_loss']

    def configure_optimizers(self):
        return torch.optim.Adam(self.fc.parameters(), lr=self.learning_rate)

# Train

## Constants

In [ ]:
# change the following paths according to output paths used in preprocessing.ipynb
DATASET_ROOT_DIR = input('Dataset root directory: ') or '/home/studio-lab-user/Dataset'

TRAIN_DIR = os.path.join(DATASET_ROOT_DIR, 'melspec_resnet50_training')
TRAIN_DIR_MIX_2 = os.path.join(DATASET_ROOT_DIR, 'melspec_resnet50_training_mix_2')
TRAIN_DIR_MIX_3 = os.path.join(DATASET_ROOT_DIR, 'melspec_resnet50_training_mix_3')
VAL_DIR = os.path.join(DATASET_ROOT_DIR, 'melspec_resnet50_validation')

## Dataset loading

In [ ]:
ds_train_raw = TrainingDataset(TRAIN_DIR)
ds_train_mix_2 = TensorDataset(TRAIN_DIR_MIX_2)
ds_train_mix_3 = TensorDataset(TRAIN_DIR_MIX_3)
ds_train = Cache(source_dataset=ConcatDataset((ds_train_raw, ds_train_mix_2,
                                               ds_train_mix_3)),
                 calculate_label_distributions=True)

In [ ]:
print('Label distribution on training data:')
print(ds_train.label_distributions)

In [ ]:
ds_provided_validation = Cache(source_dataset=TensorDataset(VAL_DIR),
                               calculate_label_distributions=True)
print('Label distribution on provided validation data:')
print(ds_provided_validation.label_distributions)

In [ ]:
ds = ds_train
ds_train, ds_validation = torch.utils.data.random_split(ds, (0.85, 0.15))

## Training

In [ ]:
# change to ds_provided_validation.label_distributions if you want to adjust
# loss according to provided validation data instead of our training data
loss = WeightedBCELoss(ds.label_distributions)
# uncomment if you want to use the default BCELoss
# loss = nn.BCELoss()

In [ ]:
train_loader = torch.utils.data.DataLoader(ds_train, batch_size=64,
                                           shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(ds_validation, batch_size=64,
                                         shuffle=True, num_workers=4)

model = LastLayer(loss=loss, num_features=2048)
model_label = input('Model label (for tensorboard graphs): ')
logger = TensorBoardLogger(save_dir=os.getcwd(), name='lightning_logs',
                           version=model_label)

trainer = pl.Trainer(max_epochs=22, logger=logger)
trainer.fit(model=model, train_dataloaders=train_loader,
            val_dataloaders=val_loader)